In [1]:
import numpy as np

# Data and model set 

In [7]:
data = np.load('data/lab2_data.npz', allow_pickle=True)['data']
phoneHMMs = np.load('data/lab2_models_onespkr.npz', allow_pickle=True)['phoneHMMs'].item()

In [8]:
list(sorted(phoneHMMs.keys()))

['ah',
 'ao',
 'ay',
 'eh',
 'ey',
 'f',
 'ih',
 'iy',
 'k',
 'n',
 'ow',
 'r',
 's',
 'sil',
 'sp',
 't',
 'th',
 'uw',
 'v',
 'w',
 'z']

In [9]:
phoneHMMs['ah'].keys()

dict_keys(['name', 'startprob', 'transmat', 'means', 'covars'])